<a href="https://colab.research.google.com/github/indipanesar96/neural-style-tf/blob/master/NN%20For%20Returns%20Big%20Data2%20CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#code32and18.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf # to create neural networks
import numpy as np # to manipulate arrays
import matplotlib.pyplot as plt # for plots
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_normal



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# This is just to allow colab to read the file, when we submit we wont need this
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# The shared link for the csv file
link = 'https://drive.google.com/open?id=1_5T1cpgLHuo6tQYhW4oi4zJ_bYKCjGq5'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('CPZ_data_mini.csv')  
df = pd.read_csv('CPZ_data_mini.csv') #this is the only line we need from this block when submitting

In [0]:



df.sort_values(by=['permno','Date'],inplace=True)

df.head()

df['ret_future'] = df.groupby('permno')['ret'].shift(-1)

df = df.dropna()
y = df['ret_future']
X = df.drop(columns=['ret','ret_future','Date','permno']).copy()

train = ('1967-01-01' <= df['Date']) & (df['Date'] < '1997-01-01')
vali = ('1997-01-01' <= df['Date']) & (df['Date'] < '2007-01-01')
test = ('2007-01-01' <= df['Date']) & (df['Date'] < '2016-11-01')


In [0]:
df

,Date,permno,ret,ST_REV,SUV,r12_2,NOA,SGA2S,LME,RNA,LTurnover,Lev,Resid_Var,D12,E12,b/m,ntis,tbl,tms,dfy,svar,ret_future
0,1967-01-01,10006,0.136223,-0.241259,-0.325175,-0.043124,0.134033,-0.383450,0.010490,0.045455,0.059441,-0.010490,-0.224942,2.880000,5.51667,0.533363,0.015698,0.0496,-0.0048,0.0077,0.000733,-0.039703
430,1967-02-01,10006,-0.039703,0.071096,-0.374126,-0.199301,0.136364,-0.383450,0.019814,0.040793,-0.008159,-0.012821,-0.264569,2.890000,5.48333,0.540048,0.017701,0.0472,-0.0007,0.0079,0.000412,0.152727
860,1967-03-01,10006,0.152727,-0.269767,0.179070,-0.155814,0.137209,-0.383721,0.004651,0.039535,0.069767,-0.011628,-0.248837,2.900000,5.45000,0.549551,0.014922,0.0456,-0.0001,0.0072,0.000501,0.048883
1291,1967-04-01,10006,0.048883,0.389655,0.428736,-0.182759,0.134483,-0.385057,0.031034,0.042529,0.214943,-0.010345,0.067816,2.900000,5.41000,0.530517,0.015046,0.0426,0.0051,0.0072,0.000837,-0.022112
1727,1967-05-01,10006,-0.022112,0.101382,-0.029954,0.050691,0.133641,-0.382489,0.034562,0.039171,0.013825,-0.006912,0.101382,2.900000,5.37000,0.558201,0.011465,0.0384,0.0098,0.0072,0.000890,0.058524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194906,2015-12-01,93433,-0.264806,0.410389,0.485229,0.500000,-0.465534,0.475382,-0.352289,-0.477351,0.498523,-0.337518,0.499015,43.387887,86.53000,0.313649,-0.021620,0.0012,0.0256,0.0149,0.002839,-0.268100
1196929,2016-01-01,93433,-0.268100,-0.453511,-0.096439,0.500000,-0.465381,0.476756,-0.380317,-0.477745,0.491098,-0.337290,0.499011,43.550571,86.50000,0.331911,-0.020272,0.0023,0.0213,0.0145,0.004301,0.162368
1198942,2016-02-01,93433,0.162368,-0.448310,-0.336481,0.500000,-0.465706,0.477137,-0.396123,-0.477634,0.302187,-0.339463,0.419483,43.713256,86.47000,0.330902,-0.024034,0.0026,0.0191,0.0138,0.002602,-0.074231
1200946,2016-03-01,93433,-0.074231,0.388667,0.263355,0.500000,-0.465552,0.476535,-0.382676,-0.477534,0.490514,-0.338742,0.351722,43.875940,86.44000,0.327955,-0.023008,0.0031,0.0187,0.0131,0.001274,-0.101623


In [0]:

##########################################
#normalized

X -= X.loc[train].mean()
X /= X.loc[train].std()


In [0]:

##########################################
#train
X[train],y[train]
#vali
X[vali],y[vali]


(          ST_REV       SUV     r12_2  ...       tms       dfy      svar
 622823 -1.268085 -0.964422 -0.828734  ...  0.015681 -0.995991 -0.080044
 625483 -1.568396  0.081883 -1.196952  ... -0.031046 -1.089615 -0.052961
 628151  0.927439 -0.732485 -1.409699  ...  0.175890 -1.089615  0.071436
 630825  1.330903  0.095651 -1.340994  ... -0.031046 -1.136427  0.308451
 633526  0.104975 -1.026207 -1.025013  ... -0.071098 -1.113021  0.072838
 ...          ...       ...       ...  ...       ...       ...       ...
 710284  0.933927 -0.418274 -0.083262  ... -0.011020 -0.808741  0.243462
 713005  0.215012  0.161204 -0.382253  ... -0.144527 -0.668304  0.237573
 715719 -1.290098  1.416624 -0.261127  ... -0.084449 -0.621492  0.750299
 718422 -0.172253 -0.758604 -0.047144  ... -0.131177 -0.715117 -0.081420
 721118  1.186107  0.096470 -0.172770  ... -0.137852 -1.066209 -0.129951
 
 [320245 rows x 18 columns], 622823    0.043719
 625483    0.071458
 628151   -0.004300
 630825    0.248621
 633526    0.0

In [0]:

#build netural networking
#32 16 8 
def train_lr32(X, y, l_r):
    
    lamda = 0 # we don't regulariziation firstly
    p = 0.1
    model = Sequential()
    model.add(Dense(32, activation='relu',
                    kernel_regularizer=l2(l=lamda), 
                    kernel_initializer=he_normal(),
                    input_dim=18))
    model.add(Dropout(p))
    model.add(Dense(16, activation='relu',
                    kernel_regularizer=l2(l=lamda),
                    kernel_initializer=he_normal()))
    model.add(Dropout(p))
    model.add(Dense(8, activation='relu',
                    kernel_regularizer=l2(l=lamda),
                    kernel_initializer=he_normal()))
    model.add(Dropout(p))
    model.add(Dense(1, activation='linear',
                    kernel_initializer=he_normal()))
    
    model.compile(loss='mse',
                  optimizer=Adam(lr=l_r, beta_1=0.9, beta_2=0.999),
                  metrics=['mse'])
    
    
    history = model.fit(X,y,epochs=20, batch_size=64,verbose=1)
    
    return model, history

#18 18 18 18
def train_lr_18ex(X, y, l_r):
    lamda = 0 # we don't regulariziation firstly
    p = 0.1
    model = Sequential()
    model.add(Dense(18, activation='relu',
                    kernel_regularizer=l2(l=lamda), 
                    kernel_initializer=he_normal(),
                    input_dim=18))
    model.add(Dropout(p))
    model.add(Dense(18, activation='relu',
                    kernel_regularizer=l2(l=lamda),
                    kernel_initializer=he_normal()))
    model.add(Dropout(p))
    model.add(Dense(18, activation='relu',
                    kernel_regularizer=l2(l=lamda),
                    kernel_initializer=he_normal()))
    model.add(Dropout(p))
    model.add(Dense(18, activation='relu',
                    kernel_regularizer=l2(l=lamda),
                    kernel_initializer=he_normal()))
    model.add(Dropout(p))
    model.add(Dense(1, activation='linear',
                    kernel_initializer=he_normal()))
    
    model.compile(loss='mse',
                  optimizer=Adam(lr=l_r, beta_1=0.9, beta_2=0.999),
                  metrics=['mse'])
    
    
    history = model.fit(X,y,epochs=20, batch_size=64,verbose=1)
    
    return model, history



In [0]:

train_X = X[train]
train_y = y[train]    
l_r_list = [0.001,0.005,0.01,0.05]
loss=[0,0,0,0]
metrics=[0,0,0,0]

#different learning rate in 32 model
model1, history1 = train_lr32(train_X, train_y, 0.001)
model2, history2 = train_lr32(train_X, train_y, 0.005)
model3, history3 = train_lr32(train_X, train_y, 0.01)
model4, history4 = train_lr32(train_X, train_y, 0.05)
#plot



Epoch 1/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0374 - mse: 0.0374
Epoch 2/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 3/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 4/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 5/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 6/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 7/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 8/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 9/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 10/20
9635/9635 [==============================] - 15s 2ms/step - loss: 0.0211 - mse: 0.0211
Epoch 11/20
9635/96

In [0]:

y0_001, = plt.plot(history1.history['loss'],label = '$0.001$')
y0_005, = plt.plot(history2.history['loss'],label = '$0.005$')
y0_01,  = plt.plot(history3.history['loss'],label = '$0.01$')
y0_05,  = plt.plot(history4.history['loss'],label = '$0.05$')
plt.legend(handles=[y0_001,y0_005, y0_01, y0_05]) 
#18*4 model




In [0]:
model14, history14 = train_lr_18ex(train_X, train_y, 0.001)
history14.history['loss']
#compare
history1.history['loss']
